This notebook gives a summary of the NIRISS commissioning data

In [ ]:
from pathlib import Path
import pandas as pd
from astropy.io import fits

First, we just get a list of all simulation files.

In [ ]:
data_dir = Path(Path.home(), "cap_simulations")
data_files = list(data_dir.iterdir())

# Summary and example header
We can first take a look at the info and header of an example file.

In [ ]:
hdul = fits.open(data_files[5])

In [ ]:
# General info
hdul.info()

In [ ]:
# Primary header
hdul[0].header

In [ ]:
# SCI header
hdul[1].header

In [ ]:
# Close the file
hdul.close()

The primary header has a lot of useful info on the observation itself and on
the instrument configuration (exposure, dither, object name, date, program, etc.).

I also printed the SCI header (hdul[1].header) because it seems important to orient the image properly
Here are a few useful links (I think):
- https://docs.astropy.org/en/stable/wcs/index.html
- https://docs.astropy.org/en/stable/api/astropy.coordinates.SkyCoord.html
- https://jwst-docs.stsci.edu/jwst-observatory-hardware/jwst-observatory-coordinate-system-and-field-of-regard
- https://jwst-docs.stsci.edu/jwst-observatory-functionality/jwst-position-angles-ranges-and-offsets

# Dataset summary
Now we can load all primary headers to get a summary of the observations.
We use a pandas dataframe to easily group and filter information.

In [ ]:
d = []
for fpath in data_files:
    hdr = fits.getheader(fpath)
    # remove blank keys from header
    del hdr['']
    d.append(dict(hdr))

df = pd.DataFrame(d)

In [ ]:
# Example quick filter
# df_filters = (df.SUBARRAY == "SUB80")
# df = df[df_filters]

# This grouby can be used to see how many files we have, broken down in
# detailed catergories
# We can comment categories if we want to simplify the summary
group_cat = [
    "OBSLABEL",
    "TARGPROP",
    "FILTER",
    "PUPIL",
    "EXP_TYPE",
    "SUBARRAY",
    "PATT_NUM",
    "SUBPXPTS",
    "NINTS",
    "NGROUPS"
]
cat_sizes = df.groupby(group_cat).size()
cat_sizes.to_frame(name="NFILES")

In [ ]:
# Now we can use the above categories as index to get files for each of them
pd.set_option("display.max_rows", None)
cat_df = df.set_index(group_cat)
cat_files = cat_df.FILENAME
cat_files.to_frame()

Some observations on the above file info:
- In Observation 1, The two files with 29 integrations per exposure in F480M at the end of observation 1 are used to reach the 72k limit, TA SNR is 286
- Observation 2 is the same as 1, but no extra F480M

Here's how the files are links (for the parts I think I understand):
- 1 and 2 have 4 and 6 as calibrators
- 3 has 5 and 7 as calibrators
    - Not sure why 7 has 25 subpixel dithers